In [25]:
#import
import numpy as np
import xarray as xr
import glob
# from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import cmocean
from netCDF4 import Dataset as NetCDFFile
from datetime import datetime, timedelta
from geopy.distance import geodesic
import os
import matplotlib
import pandas as pd
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [26]:
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
warnings.filterwarnings("ignore",category =RuntimeWarning)

In [27]:
ds =  xr.load_dataset('/Users/kat/Desktop/Brain/Area/Research/Siegelman_Lab/Data/seal_data/seal_data.nc') #put the path where you save the data

In [28]:
## select the tag of interest (e.g., all tags starting with ct158 are winter tags)
name = 'ft22-876-18'
#ft22-876-18
#ft22-878-18
#ft22-879-18
ds_tag = ds.where(ds.coords['name']==name, drop =True)

In [29]:
# create a directory where you will save your files
dir_out=f'/Users/kat/Desktop/Brain/Area/Research/Siegelman_Lab/Visualizations/{name}/' ## plot directory

In [30]:
## plotting CT, SA, B
distcum = ds_tag.time.shape[0]
dist_plot = np.hstack((np.arange(0,distcum,500),int(distcum))) ## dividing the trajectory in 500 km-segments
print(distcum)
print(dist_plot)

5120
[   0  500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5120]


In [31]:
ds_tag

<xarray.Dataset> Size: 200MB
Dimensions:       (depth: 486, time: 5120)
Coordinates:
  * time          (time) datetime64[ns] 41kB 2018-10-20T11:36:01 ... 2019-01-...
  * depth         (depth) int64 4kB 15 16 17 18 19 20 ... 496 497 498 499 500
    lat           (time) float64 41kB -49.39 -49.39 -49.39 ... -49.58 -49.58
    lon           (time) float64 41kB 70.49 70.5 70.51 ... 70.58 70.57 70.56
    name          (time) <U16 328kB 'ft22-876-18' ... 'ft22-876-18'
Data variables: (12/18)
    temperature   (depth, time) float64 20MB 3.075 2.967 2.911 ... nan nan nan
    salinity      (depth, time) float64 20MB 33.63 33.63 33.64 ... nan nan nan
    sla           (time) float64 41kB 0.0393 0.03696 0.0366 ... 0.1148 0.1147
    eke           (time) float64 41kB 0.003129 0.007194 ... 0.01415 0.01434
    ke_neuro      (time) float64 41kB 0.005852 0.005852 ... 0.01184 0.012
    sla_neuro     (time) float64 41kB 0.03896 0.03896 0.03896 ... 0.1356 0.1358
    ...            ...
    N2            (depth, time) float64 20MB nan nan nan nan ... nan nan nan nan
    MLD           (time) float64 41kB nan nan nan nan nan ... nan nan nan nan
    bx            (depth, time) float64 20MB -9.194e-08 -1.013e-07 ... nan nan
    M4            (depth, time) float64 20MB 8.453e-15 1.026e-14 ... nan nan
    qErtel        (depth, time) float64 20MB nan nan nan nan ... nan nan nan nan
    inv_Ri        (depth, time) float64 20MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [32]:
# print(ds_tag['time'][dist_plot[2]].values)[:10]

In [33]:
plt.ioff()

for i in range(len(dist_plot)-1):
    pad = 0.4

    plt.rcParams['font.size'] = '18'
    fig = plt.figure(figsize=(25,30))
    if i < len(dist_plot)-2:
        plt.suptitle(name+' \n' + str(ds_tag['time'][dist_plot[i]].values)[:10]+'/'+str(ds_tag['time'][dist_plot[i+1]].values)[:10])
    else:
        plt.suptitle(name+' \n' + str(ds_tag['time'][dist_plot[i]].values)[:10]+'/'+str(ds_tag['time'][dist_plot[i]].values)[:10])
    ax0 = plt.subplot(611)
    plt.annotate("a", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    plt.plot(ds_tag.sla_neuro, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.ylabel('SLA [m]')
    divider1 = make_axes_locatable(ax0)
    cax1 = divider1.append_axes('right', size="1%", pad=pad)
    cax1.axis('off')

    # print(i)
    ax1 = plt.subplot(612)
    plt.annotate("b", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    img = plt.imshow(ds_tag['temperature']*ds_tag['maskprof'], origin ='lower', cmap = cmocean.cm.thermal, aspect='auto',vmin = np.nanmin(ds_tag['temperature'].values[:,int(dist_plot[i]):int(dist_plot[i+1])])*0.85 ,vmax = np.nanmax(ds_tag['temperature'].values[:,int(dist_plot[i]):int(dist_plot[i+1])])*0.85)
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.ylabel('Depth [m]')
    plt.gca().invert_yaxis()
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'CT [$\degree$C]')#, format='%0.e')

    ax2 = plt.subplot(613)
    plt.annotate("c", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    cm = np.nanmean(ds_tag['temperature'][:,int(dist_plot[i]):int(dist_plot[i+1])],axis = 1)
    cm = cm[:,np.newaxis]
    cm = cm*np.ones(ds_tag['temperature'].shape[1])
    img = plt.imshow((ds_tag['temperature']-cm)*ds_tag['maskprof'], origin ='lower', cmap = 'RdBu_r', aspect='auto',vmin =  -np.nanmax(ds_tag['temperature'].values[:,int(dist_plot[i]):int(dist_plot[i+1])]-cm[:,int(dist_plot[i]):int(dist_plot[i+1])])*0.8,vmax = np.nanmax(ds_tag['temperature'].values[:,int(dist_plot[i]):int(dist_plot[i+1])]-cm[:,int(dist_plot[i]):int(dist_plot[i+1])])*0.8)
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.ylabel('Depth [m]')
    plt.gca().invert_yaxis()
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'CT$^{\prime}$ [$\degree$C]')#, format='%0.e')

    ax3 = plt.subplot(614)
    plt.annotate("c", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    img = plt.imshow(ds_tag['salinity']*ds_tag['maskprof'], origin ='lower', cmap = cmocean.cm.haline, aspect='auto', vmin = np.nanmin(ds_tag['salinity'].values[:,int(dist_plot[i]):int(dist_plot[i+1])]) ,vmax = np.nanmax(ds_tag['salinity'].values[:,int(dist_plot[i]):int(dist_plot[i+1])]))
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.ylabel('Depth [m]')
    plt.gca().invert_yaxis()
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'SA [PSU]')#, format='%0.e')   

    ax4 = plt.subplot(615)
    plt.annotate("d", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    sm = np.nanmean(ds_tag['salinity'][:,int(dist_plot[i]):int(dist_plot[i+1])],axis = 1)
    sm = sm[:,np.newaxis]
    sm = sm*np.ones(ds_tag['salinity'].shape[1])
    img = plt.imshow((ds_tag['salinity']-sm)*ds_tag['maskprof'], origin ='lower', cmap = 'RdBu_r', aspect='auto', vmin = -np.nanmax(ds_tag['salinity'][:,int(dist_plot[i]):int(dist_plot[i+1])]-sm[:,int(dist_plot[i]):int(dist_plot[i+1])])*0.8,vmax = np.nanmax(ds_tag['salinity'][:,int(dist_plot[i]):int(dist_plot[i+1])]-sm[:,int(dist_plot[i]):int(dist_plot[i+1])])*0.8)
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.ylabel('Depth [m]')
    plt.gca().invert_yaxis()
    divider = make_axes_locatable(ax4)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'SA$^{\prime}$ [PSU]')#, format='%0.e') 

    ax5 = plt.subplot(616)
    plt.annotate("e", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    img = plt.imshow(ds_tag['buoyancy']*ds_tag['maskprof'], origin ='lower', cmap = cmocean.cm.dense_r, aspect='auto',vmin = np.nanmin(ds_tag['buoyancy'].values[:,int(dist_plot[i]):int(dist_plot[i+1])])*0.95,vmax = np.nanmax(ds_tag['buoyancy'].values[:,int(dist_plot[i]):int(dist_plot[i+1])])*1.05)
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.xlabel('Along-track distance [km]')
    plt.ylabel('Depth [m]')
    plt.gca().invert_yaxis()
    divider = make_axes_locatable(ax5)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'b [m.s$^{-2}$]')#, format='%0.e') 

    save_dir = '/Users/kat/Desktop/Siegelman_Lab/Images/'+name+'/'
    print(save_dir)

    plt.savefig('//Users/kat/Desktop/Brain/Area/Research/Siegelman_Lab/Visualizations/ft22-876-18/Images/TEMP/' +str(ds_tag['time'][dist_plot[i]].values)[:10])


/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/
/Users/kat/Desktop/Siegelman_Lab/Images/ft22-876-18/


In [34]:
# plt.ioff()
# dist_plot = np.hstack((np.arange(0,distcum,500),int(distcum))) ## dividing the trajectory in 500 km-segments
# for i in range(len(dist_plot)-1):
#     pad = 0.4
#     plt.rcParams['font.size'] = '18'
#     fig = plt.figure(figsize=(25,30))
#     if i < len(dist_plot)-2:
#         plt.suptitle(name+' \n' + str(ds_tag['time'][dist_plot[i]].values)[:10]+'/'+str(ds_tag['time'][dist_plot[i+1]].values)[:10])
#     else:
#         plt.suptitle(name+' \n' + str(ds_tag['time'][dist_plot[i]].values)[:10]+'/'+str(ds_tag['time'][dist_plot[i]].values)[:10])
        
#     ax1 = plt.subplot(611)
#     plt.annotate("a", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
#     plt.plot(ds_tag.sla_neuro, c='k',lw = 2, label = 'SLA [m]')
#     plt.plot(ds_tag.eke_neuro, c='r',lw = 2, label = 'EKE [m$^2$.s$^-2$]')
#     plt.plot(ds_tag.strain_neuro*1e4, c='b',lw = 2, label = '$10^4 \sigma$ [s$^{-1}]$')
#     plt.plot(ds_tag.zeta_neuro*1e4, c='orange',lw = 2, label = '$10^4 \zeta$ [s$^{-1}]$')
#     # plt.plot(ds_tag.zeta_neuro*1e4, c='orange',lw = 2, label = r'$\zeta \times 10^{4} \mathrm{s}^{-1}')
#     plt.xlim((dist_plot[i],dist_plot[i+1]))
#     # plt.ylabel('SLA [m]')
#     plt.legend()
#     divider1 = make_axes_locatable(ax1)
#     cax1 = divider1.append_axes('right', size="1%", pad=pad)
#     cax1.axis('off')
    

#     ax = plt.subplot(612)
#     plt.annotate("b", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
#     bm = np.nanmean(ds_tag['buoyancy'][:,int(dist_plot[i]):int(dist_plot[i+1])],axis = 1)
#     bm = bm[:,np.newaxis]
#     bm = bm*np.ones(ds_tag['buoyancy'].shape[1])
#     img = plt.imshow((ds_tag['buoyancy']-bm)*ds_tag['maskprof'], origin ='lower', cmap = 'RdBu_r', aspect='auto',vmin = -2e-3,vmax = 2e-3)
#     plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
#     plt.xlim((dist_plot[i],dist_plot[i+1]))
#     plt.ylabel('Depth [m]')
#     plt.gca().invert_yaxis()
#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes('right', size="1%", pad=pad)
#     cbar = plt.colorbar(img, cax=cax,label = r'b$^{\prime}$ [m.s$^{-2}$]', format='%0.e')

#     ax2 = plt.subplot(613)
#     plt.annotate("c", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
#     img = plt.imshow(ds_tag['bx']*ds_tag['maskprof'], origin ='lower', cmap = 'RdBu_r', vmin = -2e-7,vmax = 2e-7, aspect='auto')
#     plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
#     plt.xlim((dist_plot[i],dist_plot[i+1]))
#     plt.ylabel('Depth [m]')
#     plt.gca().invert_yaxis()
#     # plt.colorbar( shrink=0.9,pad = 0.025, label = r'b$_x$ [s$^{-2}$]')
#     divider = make_axes_locatable(ax2)
#     cax = divider.append_axes('right', size="1%", pad=pad)
#     cbar = plt.colorbar(img, cax=cax,label = r'b$_x$ [s$^{-2}$]')

#     ax3 = plt.subplot(614)
#     plt.annotate("d", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
#     img = plt.imshow(ds_tag['qErtel']*ds_tag['maskprof'], origin ='lower', cmap = 'RdBu_r', aspect='auto',vmin = -1e-9, vmax = 1e-9)
#     plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
#     plt.xlim((dist_plot[i],dist_plot[i+1]))
#     plt.ylabel('Depth [m]')
#     plt.ylim((15,486))
#     plt.gca().invert_yaxis()
#     # plt.colorbar( shrink=0.9,pad = 0.025, label = r'Ertel PV [s$^{-3}$]')
#     divider = make_axes_locatable(ax3)
#     cax = divider.append_axes('right', size="1%", pad=pad)
#     cbar = plt.colorbar(img, cax=cax,label = r'Ertel PV [s$^{-3}$]')

#     ax4 = plt.subplot(615)
#     plt.annotate("e", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
#     # plt.title(name[66:-12])
#     img = plt.imshow(ds_tag['N2']*ds_tag['maskprof'], origin ='lower', cmap = 'Blues', vmin=0,vmax=1e-5, aspect='auto')
#     plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
#     plt.xlim((dist_plot[i],dist_plot[i+1]))
#     # plt.xlabel('Along-track distance [km]')
#     plt.ylabel('Depth [m]')
#     plt.ylim((15,486))
#     plt.gca().invert_yaxis()
#     # plt.colorbar( shrink=0.9,pad = 0.025, label = r'N$^2$ [s$^{-2}$]')
#     divider = make_axes_locatable(ax4)
#     cax = divider.append_axes('right', size="1%", pad=pad)
#     cbar = plt.colorbar(img, cax=cax,label = r'N$^2$ [s$^{-2}$]')

#     ax5 = plt.subplot(616)
#     plt.annotate("f", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
#     # plt.title(name[66:-12])
#     img = plt.imshow(ds_tag['inv_Ri']*ds_tag['maskprof'], origin ='lower', cmap = 'Blues_r',norm = LogNorm(vmin = 5e-3,vmax = 1e1), aspect='auto')
#     plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
#     plt.xlim((dist_plot[i],dist_plot[i+1]))
#     plt.xlabel('Along-track distance [km]')
#     plt.ylabel('Depth [m]')
#     plt.ylim((15,486))
#     plt.gca().invert_yaxis()
#     # plt.colorbar( shrink=0.9,pad = 0.025, label = r'Ri$^{-1}$')
#     divider = make_axes_locatable(ax5)
#     cax = divider.append_axes('right', size="1%", pad=pad)
#     cbar = plt.colorbar(img, cax=cax,label = r'Ri$^{-1}$')


#     plt.savefig('/Users/kat/Desktop/Brain/Area/Research/Siegelman_Lab/Visualizations/ct158-199-19/Images/ERTEL/' +str(ds_tag['time'][dist_plot[i]].values)[:10])

In [41]:
plt.ioff()
dist_plot = np.hstack((np.arange(0, distcum, 500), int(distcum)))  # dividing the trajectory in 500 km-segments
for i in range(len(dist_plot)-1):
    pad = 0.4
    plt.rcParams['font.size'] = '18'
    fig = plt.figure(figsize=(25, 30))
    if i < len(dist_plot)-2:
        plt.suptitle(name+' \n' + str(ds_tag['time'][dist_plot[i]].values)[:10]+'/'+str(ds_tag['time'][dist_plot[i+1]].values)[:10])
    else:
        plt.suptitle(name+' \n' + str(ds_tag['time'][dist_plot[i]].values)[:10]+'/'+str(ds_tag['time'][dist_plot[i]].values)[:10])

    ax1 = plt.subplot(611)
    plt.annotate("a", xy=(0, 1.02), xycoords="axes fraction", weight='bold', size=24)
    ax1.plot(ds_tag.sla_neuro, c='k', lw=2, label='SLA [m]')

    # Create a second y-axis for EKE
    ax2 = ax1.twinx()  # This creates a second y-axis on the right
    ax2.plot(ds_tag.strain_neuro, c='r', lw=2, label='Strain/f', linestyle='--')
    # ax2.set_ylim(-0.2,2)
    # ax1.set_ylim(-0.2,0.4)
    ax1.set_xlim((dist_plot[i], dist_plot[i+1]))

    ax1.set_ylabel('SLA [m]')
    ax2.set_ylabel('Strain/f [m².s⁻²]', color='r')

    # Customizing the ticks of the second axis
    ax2.tick_params(axis='y', labelcolor='r')

    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')


    ax = plt.subplot(612)
    plt.annotate("b", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    bm = np.nanmean(ds_tag['buoyancy'][:,int(dist_plot[i]):int(dist_plot[i+1])],axis = 1)
    bm = bm[:,np.newaxis]
    bm = bm*np.ones(ds_tag['buoyancy'].shape[1])
    img = plt.imshow((ds_tag['buoyancy']-bm)*ds_tag['maskprof'], origin ='lower', cmap = 'RdBu_r', aspect='auto',vmin = -2e-3,vmax = 2e-3)
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.ylabel('Depth [m]')
    plt.gca().invert_yaxis()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'b$^{\prime}$ [m.s$^{-2}$]', format='%0.e')

    ax2 = plt.subplot(613)
    plt.annotate("c", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    img = plt.imshow(ds_tag['bx']*ds_tag['maskprof'], origin ='lower', cmap = 'RdBu_r', vmin = -2e-7,vmax = 2e-7, aspect='auto')
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.ylabel('Depth [m]')
    plt.gca().invert_yaxis()
    # plt.colorbar( shrink=0.9,pad = 0.025, label = r'b$_x$ [s$^{-2}$]')
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'b$_x$ [s$^{-2}$]')

    ax3 = plt.subplot(614)
    plt.annotate("d", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    img = plt.imshow(ds_tag['qErtel']*ds_tag['maskprof'], origin ='lower', cmap = 'RdBu_r', aspect='auto',vmin = -1e-9, vmax = 1e-9)
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.ylabel('Depth [m]')
    plt.ylim((15,486))
    plt.gca().invert_yaxis()
    # plt.colorbar( shrink=0.9,pad = 0.025, label = r'Ertel PV [s$^{-3}$]')
    divider = make_axes_locatable(ax3)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'Ertel PV [s$^{-3}$]')

    ax4 = plt.subplot(615)
    plt.annotate("e", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    # plt.title(name[66:-12])
    img = plt.imshow(ds_tag['N2']*ds_tag['maskprof'], origin ='lower', cmap = 'Blues', vmin=0,vmax=1e-5, aspect='auto')
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    # plt.xlabel('Along-track distance [km]')
    plt.ylabel('Depth [m]')
    plt.ylim((15,486))
    plt.gca().invert_yaxis()
    # plt.colorbar( shrink=0.9,pad = 0.025, label = r'N$^2$ [s$^{-2}$]')
    divider = make_axes_locatable(ax4)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'N$^2$ [s$^{-2}$]')

    ax5 = plt.subplot(616)
    plt.annotate("f", xy=(0, 1.02), xycoords="axes fraction",weight='bold', size = 24)
    # plt.title(name[66:-12])
    img = plt.imshow(ds_tag['inv_Ri']*ds_tag['maskprof'], origin ='lower', cmap = 'Blues_r',norm = LogNorm(vmin = 5e-3,vmax = 1e1), aspect='auto')
    plt.plot(ds_tag['MLD'].values-15, c='k',lw = 2)
    plt.xlim((dist_plot[i],dist_plot[i+1]))
    plt.xlabel('Along-track distance [km]')
    plt.ylabel('Depth [m]')
    plt.ylim((15,486))
    plt.gca().invert_yaxis()
    # plt.colorbar( shrink=0.9,pad = 0.025, label = r'Ri$^{-1}$')
    divider = make_axes_locatable(ax5)
    cax = divider.append_axes('right', size="1%", pad=pad)
    cbar = plt.colorbar(img, cax=cax,label = r'Ri$^{-1}$')


    plt.savefig('/Users/kat/Desktop/Brain/Area/Research/Siegelman_Lab/Visualizations/ft22-876-18/Images/ERTEL/' +str(ds_tag['time'][dist_plot[i]].values)[:10])